#importing_necessary_libraries

In [5]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
!pip install tkan
from tkan import TKAN
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import mean_squared_error, r2_score

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.9/60.9 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 3.8 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
contourpy 1.2.0 requires numpy<2.0,>=1.20, but you have numpy 2.0.0 which is incompatible.
tensorflow 2.17.0 requires numpy<2.0.0,>=1.26.0; python_version >= "3.12", but you have numpy 2.0.0 which is incompatible.
streamlit 1.32.0 requires numpy<2,>=1.19.3, but you have numpy 2.0.0 which is incompatible.
numba 0.59.1 requires numpy<1.27,>=1.22, but you have numpy 2.0.0 which is incompatible.
pywavelets 1.5.0 requires numpy<2.0,>=1.22.4, but you have numpy 2.0.0 which is incompatible.


In [11]:
import tensorflow as tf
import random
seed=42
np.random.seed(seed)
tf.random.set_seed(seed)
random.seed(seed)

In [13]:
data=pd.read_csv('Datos históricos del S&P 500.csv')
data

,Fecha,Último,Apertura,Máximo,Mínimo,Vol.,% var.
0,05.11.2024,"5.782,76","5.722,43","5.783,44","5.722,10",NaN,"1,23%"
1,04.11.2024,"5.712,69","5.725,15","5.741,43","5.696,51",NaN,"-0,28%"
2,01.11.2024,"5.728,80","5.723,22","5.772,52","5.723,22",NaN,"0,41%"
3,31.10.2024,"5.705,45","5.775,34","5.775,34","5.702,86",NaN,"-1,86%"
4,30.10.2024,"5.813,67","5.832,65","5.850,94","5.811,28",NaN,"-0,33%"
5,29.10.2024,"5.832,92","5.819,68","5.847,19","5.802,17",NaN,"0,16%"
6,28.10.2024,"5.823,52","5.833,93","5.842,92","5.823,08",NaN,"0,27%"
7,25.10.2024,"5.808,12","5.826,75","5.862,82","5.799,98",NaN,"-0,03%"
8,24.10.2024,"5.809,86","5.817,80","5.817,80","5.784,92",NaN,"0,21%"
9,23.10.2024,"5.797,42","5.834,50","5.834,85","5.762,41",NaN,"-0,92%"


In [17]:
data['Último']=data['Último'].apply(lambda x:float(str(x).replace(',' , ''))*1000)
data['Fecha']=pd.to_datetime(data['Fecha'], format="%d.%m.%Y")
data=data.sort_values(by='Fecha', ascending=True).reset_index(drop=True)


In [19]:
data_series=data['Último'].values.reshape(-1, 1)

#Data_Modelling

In [22]:
scaler=MinMaxScaler()
data_series_scaled=scaler.fit_transform(data_series)

In [41]:
data_series_scaled

array([[0.        ],
       [0.32709062],
       [0.56954898],
       [0.49848871],
       [0.70580217],
       [0.97143365],
       [0.70716529],
       [0.86842885],
       [0.86250222],
       [1.        ],
       [0.93664434],
       [0.92016832],
       [0.60143424],
       [0.6751615 ],
       [0.66484917],
       [0.75611924],
       [0.81182955],
       [0.69774195],
       [0.05636224],
       [0.19474901],
       [0.09927102],
       [0.51454987]])

In [58]:
scaler = MinMaxScaler()
data_series_scaled = scaler.fit_transform(data_series)

window_size = 5
future_steps_list = [1, 2, 3, 5]
results = []

for future_steps in future_steps_list:
    X, y = [], []
    for i in range(len(data_series_scaled) - window_size - future_steps + 1):
        X.append(data_series_scaled[i:i + window_size])
        y.append(data_series_scaled[i + window_size:i + window_size + future_steps].flatten())


X = np.array(X)
y = np.array(y)
split_index = int(len(X) * 0.8)
X_train, X_test = X[:split_index], X[split_index:]
y_train, y_test = y[:split_index], y[split_index:]
fechas_train = data['Fecha'].iloc[window_size:split_index + window_size].values
fechas_test = data['Fecha'].iloc[split_index + window_size:split_index + window_size + len(y_test)].values

In [60]:
print(X_train.shape)

(10, 5, 1)


In [62]:
y_train

array([[0.97143365, 0.70716529, 0.86842885, 0.86250222, 1.        ],
       [0.70716529, 0.86842885, 0.86250222, 1.        , 0.93664434],
       [0.86842885, 0.86250222, 1.        , 0.93664434, 0.92016832],
       [0.86250222, 1.        , 0.93664434, 0.92016832, 0.60143424],
       [1.        , 0.93664434, 0.92016832, 0.60143424, 0.6751615 ],
       [0.93664434, 0.92016832, 0.60143424, 0.6751615 , 0.66484917],
       [0.92016832, 0.60143424, 0.6751615 , 0.66484917, 0.75611924],
       [0.60143424, 0.6751615 , 0.66484917, 0.75611924, 0.81182955],
       [0.6751615 , 0.66484917, 0.75611924, 0.81182955, 0.69774195],
       [0.66484917, 0.75611924, 0.81182955, 0.69774195, 0.05636224]])

#Construction_and_training_of_TKAN_Model
The TKAN model is built and trained using a sequential architecture with a TKAN layer followed by a dense layer to multiple 
future steps. Model is compiled with Adam optimiser and a mean squared error(MSE) loss function. 

In [65]:
model_tkan=Sequential([
    TKAN(200, sub_kan_configs=[{'spline_order':4, 'grid_size':12}, {'spline_order':3, 'grid_size':10}, {'spline_order':5, 'grid_size':8}],
         return_sequences=False, use_bias=True),
    Dense(units=future_steps, activation='linear')
])
model_tkan.compile(optimizer=Adam(learning_rate=0.001), loss='mse')
history_tkan=model_tkan.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.1)

y_pred_tkan=model_tkan.predict(X_test)
y_test_inverse=scaler.inverse_transform(y_test)
y_pred_tkan_inverse=scaler.inverse_transform(y_pred_tkan)

Epoch 1/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step - loss: 1.3759 - val_loss: 0.7782
Epoch 2/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 1.0142 - val_loss: 0.5395
Epoch 3/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - loss: 0.7133 - val_loss: 0.3596
Epoch 4/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - loss: 0.4733 - val_loss: 0.2351
Epoch 5/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 0.2918 - val_loss: 0.1602
Epoch 6/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - loss: 0.1640 - val_loss: 0.1271
Epoch 7/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - loss: 0.0830 - val_loss: 0.1266
Epoch 8/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - loss: 0.0405 - val_loss: 0.1487
Epoch 9/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - loss: 0.0272 - val_loss: 0.1837
Epoch 10/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - loss: 0.0337 - val_loss: 0.2232
Epoch 11/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - loss: 0.0514 - val_loss: 0.2604
Epoch 12/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - loss: 0.0734 - val_loss: 0.2907
Epo

#Construction_and_Training_of_LSTM_model
is built and trained in similar way as TKAN , using an LSTM layer followed by a dense layer to predict multiple steps.

In [68]:
model_lstm=Sequential([
    LSTM(200, input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=False),
    Dense(units=future_steps, activation='linear')
])
model_lstm.compile(optimizer='adam', loss='mse')
history_lstm=model_lstm.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2)

y_pred_lstm=model_lstm.predict(X_test)
y_pred_lstm_inverse=scaler.inverse_transform(y_pred_lstm)

Epoch 1/50


/opt/anaconda3/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 533ms/step - loss: 0.6899 - val_loss: 0.4346
Epoch 2/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - loss: 0.6411 - val_loss: 0.3923
Epoch 3/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - loss: 0.5942 - val_loss: 0.3518
Epoch 4/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - loss: 0.5485 - val_loss: 0.3126
Epoch 5/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 0.5036 - val_loss: 0.2744
Epoch 6/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - loss: 0.4590 - val_loss: 0.2370
Epoch 7/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 0.4143 - val_loss: 0.2003
Epoch 8/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 0.3693 - val_loss: 0.1648
Epoch 9/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 0.3237 - val_loss: 0.1311
Epoch 10/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 0.2777 - val_loss: 0.1006
Epoch 11/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 0.2317 - val_loss: 0.0757
Epoch 12/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 0.1867 - val_loss: 0.0602
Epoch 13/50

#Evaluation_of_models

In [71]:
mse_tkan=mean_squared_error(y_test_inverse, y_pred_tkan_inverse)
r2_tkan=r2_score(y_test_inverse, y_pred_tkan_inverse)

mse_lstm=mean_squared_error(y_test_inverse, y_pred_lstm_inverse)
r2_lstm=r2_score(y_test_inverse, y_pred_lstm_inverse)

results.append({
    'future_steps':future_steps,
    'R^2_TKAN':r2_tkan,
    'R^2_LSTM':r2_lstm
})
results_df=pd.DataFrame(results)

In [73]:
results

[{'future_steps': 5,
  'R^2_TKAN': -37.755708830194145,
  'R^2_LSTM': -25.95765195475434}]